In [2]:
!pip install -I xpander-sdk==1.24.0 openai python-dotenv

  Using cached xpander_sdk-1.24.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached openai-1.59.6-py3-none-any.whl.metadata (27 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached jsii-1.106.0-py3-none-any.whl.metadata (79 kB)
  Using cached publication-0.0.3-py2.py3-none-any.whl.metadata (8.7 kB)
  Using cached typeguard-4.2.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached anyio-4.8.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.8.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached pydantic-2.10.5-py3-none-any.whl.metadata (30 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using 

In [12]:
from xpander_sdk import XpanderClient
from openai import OpenAI
from dotenv import load_dotenv
import os

## Load environment variables
load_dotenv()
friendli_client = OpenAI(
    base_url="https://api.friendli.ai/serverless/v1",
    api_key=os.environ.get("FRIENDLI_TOKEN")
)

xpanderAPIKey = "qYc2SlaFvE4lijE4besqT1n9UI7FKRK8W4VoFsx3"
xpanderAgentID = "e9c9244e-ef29-4ce8-87ee-b312dc69a490"


xpander_client = XpanderClient(api_key=xpanderAPIKey)
xpander_agent = xpander_client.agents.get(agent_id=xpanderAgentID)

tools = xpander_agent.retrieve_all_graph_tools()
print(tools)


[{'type': 'function', 'function': {'name': 'FirecrawlScrapingServiceExtractWebpageContent', 'description': "Extracts content from a specified webpage URL. If 'url' isn't provided, run SiteMappingRetrieveWebsiteLinks before anything else to find relevant URLs. Use this operation to gather specific webpage content, perform custom extractions, or prepare data for further analysis. Common use cases include content scraping, data extraction, and website analysis. Supports custom headers, main content extraction, and wait times for JavaScript-heavy pages. Offers LLM-based extraction with custom prompts and schemas for targeted data retrieval. Configurable actions allow interaction with dynamic web elements before content capture.", 'parameters': {'type': 'object', 'properties': {'bodyParams': {'type': 'object', 'properties': {'actions': {'type': 'array', 'description': 'Actions to perform on the page before grabbing the content', 'items': {'type': 'object', 'properties': {'amount': {'type': 

In [4]:
# verify the friendli client is working
completion = friendli_client.chat.completions.create(
    model="meta-llama-3.1-8b-instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Hello. How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [13]:
memory = [{"role": "system", "content": "You are a helpful assistant. You are running in While True Loop, the response to your tools are available, once you have completed the user request add ##FINAL ANSWER## in your reply without further function calling"},
          {"role": "user", "content": "search for xpander.ai crounchbase page and latest trends from their linkedin page "}]
print(memory[-1])
number_of_calls = 1
while True:
    tools = xpander_agent.retrieve_all_graph_tools()
    llm_response = friendli_client.chat.completions.create(
        model="meta-llama-3.1-8b-instruct",
        messages=memory,
        tools=tools,
        temperature=1
        )
    memory.append({"role": "assistant", "content": f'Step number: {number_of_calls}'})
    model_response = llm_response.choices[0].message
    print(model_response)
    memory.append({"role": "assistant", "content": model_response.content})
    if model_response.tool_calls:
        print("model decided to invoke tools")
        tools_to_run = XpanderClient.extract_tool_calls(llm_response=llm_response.model_dump())
        for tool in tools_to_run: 
            print(tool.name)
        tool_responses = xpander_agent.run_tools(tool_calls=tools_to_run)
        for tool_response in tool_responses:
            print("tool response for", tool_response.function_name, tool_response.status_code)
            memory.append({"role": "tool", "content": tool_response.result, "tool_call_id": tool_response.tool_call_id})
    if (model_response.content):
        if "##FINAL ANSWER##" in model_response.content:
            break
    number_of_calls += 1
print(model_response.content)   

{'role': 'user', 'content': 'search for xpander.ai crounchbase page and latest trends from their linkedin page '}
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_5iydQrrsp2rJKgRLDDcXc7N1', function=Function(arguments='{"bodyParams": {}, "queryParams": {"query": "xpander.ai crunchbase page"}, "pathParams": {}}', name='CrunchbaseOrganizationSearchAutocompleteByQuery'), type='function'), ChatCompletionMessageToolCall(id='call_KWgpHbhH7u9USf30lMK5V1KQ', function=Function(arguments='{"bodyParams": {}, "queryParams": {"organization_identifier": "amr6px"}, "pathParams": {}}', name='CrunchbaseOrganizationDetailsGetInfoByIdentifier'), type='function'), ChatCompletionMessageToolCall(id='call_8ejQEUbCTbDubLnPYw0L7FuK', function=Function(arguments='{"bodyParams": {"query": "xpander.ai latest trends", "search_depth": "advanced"}, "queryParams": {}, "pathParams": {}}', name='tavily-insights-fetchIn